# Projektarbeit: Absatz bei Kunden im Großhandel (Statistical Learning)
* **Ziel des Projekts**: Einteilung der Großhandelskunden in sinnvolle Gruppen mithilfe einer Clustermethode

## 1. Datenexploration
In diesem Kapitel erforscht man die Daten durch Visualisierungen, um zu verstehen, wie jedes Merkmal mit den anderen zusammenhängt. Das Ziel der Datenexploration ist es, zu sehen, ob etwas von Interesse ist.

In [ ]:
# Für dieses Projekt erforderliche Bibliotheken importieren
import sys
#!{sys.executable} -m pip install pandas-profiling
import numpy as np
import pandas as pd
from IPython.display import display 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from pandas_profiling import ProfileReport

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline

#Daten importieren
try:
    df = pd.read_csv("../input/wholesale-customers-data-set/Wholesale customers data.csv")
    print("Wholesale customers dataset has {} samples with {} features each.".format(*df.shape))
except:
    print("Dataset could not be loaded.")

In [ ]:
data = df.rename(columns={'Delicassen': 'Delicatessen'}) #Rechtschreibung korrigieren
data.drop(['Region', 'Channel'], axis = 1, inplace = True)
data.head()

In [ ]:
# allgemeine Beschreibung des Datensatzes
display(data.describe())

In [ ]:
#Ein detailierter Bericht von allen Variablen
data.profile_report()

Zuerst wollen wir *Channel* und *Region* einzig untersuchen, da die beide Merkmalen wahrscheinlich mit unserer Analyse nicht relevant sind. Es erscheint, dass es nicht so viele Unterschiede zwischen den beiden.

In [ ]:
# Channel plotten
a = df.pop('Channel')
data = data.join(a)
subjects = ['Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicatessen']
channel = data.groupby('Channel')[subjects].sum()
channel1 = channel.T
print(channel1)
colors = ['c','coral','limegreen']
ax = channel1.plot(kind='bar', figsize = (9,7),stacked= True, title = '', color = colors)
ax.tick_params(axis='x', rotation=0)
ax.legend(['Channel 1', 'Channel 2'],loc=1, prop={'size': 16})

In [ ]:
#Region plotten
a = df.pop('Region')
data = data.join(a)
region = data.groupby('Region')[subjects].sum()
region1 = region.T
print(region1)
ax = region1.plot(kind='bar', figsize = (8,6),stacked= True, title = '', color = colors)
ax.tick_params(axis='x', rotation=0)
ax.legend(['Region 1', 'Region 2','Region 3'],loc=1, prop={'size': 16})

## 1.1 Samples
Um die Daten besser zu verstehen, wäre es am besten, ein paar Beobachtungen als Stichproben auszuwählen und sie genauer zu untersuchen. Im Codeblock unten werden drei Indizes der Wahl zur Liste hinzugefügt, die die zu verfolgenden Kunden repräsentieren werden.

In [ ]:
data.drop(['Region', 'Channel'], axis = 1, inplace = True)

# drei beliebige Beobachtungen
indices = [43, 344, 122]

# Ein DataFrame für die Stichproben erzeugen
samples = pd.DataFrame(data.loc[indices], columns = data.columns)
print ("Chosen samples of wholesale customers dataset:")
display(samples)

# Durchschnitt berechnen 
mean_data = data.describe().loc['mean', :]
samples_bar = samples.append(mean_data)

# Index kontruieren
samples_bar.index = indices + ['mean']

# Barplot
ax=samples_bar.plot(kind='bar', figsize=(14,8))
ax.tick_params(rotation=0)

In [ ]:
#Vergleichen mit durchschnittlichen Wert
(samples - data.mean()) / data.std()

## 1.2 Überprüfung auf Missings und Ausreißer
Das Erkennen von Missings und Ausreißern des Datensatzes ist ein wichtiger Teil der Datenexplorationsarbeit. Wir überprüfen auf Ausreißer durch Visualisierung mittels *Pairplot*, *Barplot* und *Boxplot*.

In [ ]:
# Überprüfen, ob es Missings gibt
print(data.isnull().values.any())
# Es gibt keine Null-Werte bzw. Missings im Datensatz

In [ ]:
# Überprüfen, ob es Ausreißer gibt
_ = sns.pairplot(data, diag_kind = 'kde')

In [ ]:
plt.figure(figsize = (20,8))
_ = sns.barplot(data=data, palette="Set2")

In [ ]:
plt.figure(figsize = (20,8))
_ = sns.boxplot(data=data, orient='h', palette="Set2")

## 2. Implementierung
## 2.1 Datenverarbeitung
Vor der Clusteranalyse müssen folgende Voraussetzungen erfüllt sein:
* **Normalisierung**: Da die Daten verzehrt sind, fangen wir mit der Normalisierung an, indem wir einfach den natürlichen Logarithmus auf die Daten verwenden.
* **Ausreißer**: Es ist festgestellt, der Datensatz von vielen Ausreißer zerstört wird. Ausgewählt ist die Methode nach Tukey mit Hilfe des Interquartilabstandes, um Ausreißer zu erkennen.
* **Dimensionsreduktion**: mittels Hauptkomponentenanalyse (PCA). Sie dient dazu, umfangreiche Datensätze zu struktieren, zu vereinfachen und zu veranschaulichen.

## Normalisierung

In [ ]:
# die Daten mit dem natürlichen Logarithmus skalieren
log_data = np.log(data)

# die Stichprobedaten mit dem natürlichen Logarithmus skalieren
log_samples = np.log(samples)

# eine Streumatrix für jedes Paar neu transformierter Merkmale erstellen
_ = sns.pairplot(log_data, diag_kind = 'kde')

# die Protokolltransformierten Beispieldaten anzeigen
display(log_samples)
ax=log_samples.plot(kind='bar', figsize=(8,6))
ax.tick_params(rotation=0)
ax.set_title('Samples nach dem Log-Transformation')

## Ausreißerbehandlung

In [ ]:
outliers_list = []
# Für jedes Merkmal die Datenpunkte mit extrem hohen oder niedrigen Werten finden
for feature in log_data.keys():
    
    # Q1 (25. Perzentil der Daten) für das gegebene Merkmal berechnen
    Q1 = np.percentile(log_data[feature], 25)
    
    # Q3 (75. Perzentil der Daten) für das gegebene Merkmal berechnen
    Q3 = np.percentile(log_data[feature], 75)
    
    # den Interquartilabstand verwenden, um einen Ausreißerschritt zu berechnen (1,5-facher Interquartilabstand)
    step = (Q3 - Q1) * 1.5
    
    # Ausreißer anzeigen
    print("Data points considered outliers for the feature '{}':".format(feature))
    outliers = list(log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))].index.values)
    display(log_data[~((log_data[feature] >= Q1 - step) & (log_data[feature] <= Q3 + step))])
    outliers_list.extend(outliers)
    
print("List of Outliers -> {}".format(outliers_list))
duplicate_outliers_list = list(set([x for x in outliers_list if outliers_list.count(x) >= 2]))
duplicate_outliers_list.sort()
print("\nList of Common Outliers -> {}".format(duplicate_outliers_list))

# die Indizes für Datenpunkte auswählen, die wir entfernen möchten
outliers  = duplicate_outliers_list

# die Ausreißer entfernen, falls welche angegeben wurden
good_data = log_data.drop(log_data.index[outliers]).reset_index(drop = True)
display(good_data)

In [ ]:
# Überprüfen, ob die Stichproben Ausreißer sind
# => Sie sind nicht Ausreißer. Die obere Auswertung kann so bleiben.
print(43 in outliers_list)
print(344 in outliers_list)
print(122 in outliers_list)

## Dimensionsreduktion

In [ ]:
#Hauptkomponentenanalyse
from sklearn.decomposition import PCA

# PCA auf die guten Daten mit der gleichen Anzahl von Dimensionen wie Merkmale anwenden
pca = PCA()
pca.fit_transform(good_data)

# die gleiche PCA-Transformation auf die drei Stichproben-Datenpunkte anwenden
pca_samples = pca.transform(log_samples)

In [ ]:
# Die Varianz, die durch jede Hauptkomponente erklärt wird
explained_variances = pca.explained_variance_ratio_

print("Proportion of the variance explained by each dimension")
print("\n".join(["{}: {:1.3f}".format(i+1,val) for i,val in enumerate(explained_variances)]))

# Alle Varianz, die durch die n-te Hauptkomponente erklärt wird
cumulative_variance = [explained_variances[:i+1].sum() for i in range(len(explained_variances))]
print("\nTotal variance explained by the first N principal compoments")
print("\n".join(["{}: {:1.3f}".format(i+1,val) for i,val in enumerate(cumulative_variance)]))

In [ ]:
def pca_results(good_data, pca):
	'''
	Create a DataFrame of the PCA results
	Includes dimension feature weights and explained variance
	Visualizes the PCA results
	'''

	# Dimensionsindexierung
	dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

	# PCA Komponenten
	components = pd.DataFrame(np.round(pca.components_, 4), columns = list(good_data.keys()))
	components.index = dimensions

	# PCA explained variance
	ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
	variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
	variance_ratios.index = dimensions

	# eine Barplot-Visualisierung
	fig, ax = plt.subplots(figsize = (14,8))

	# die Merkmalsgewichte als Funktion der Komponenten plotten
	components.plot(ax = ax, kind = 'bar');
	ax.set_ylabel("Feature Weights")
	ax.set_xticklabels(dimensions, rotation=0)


	# die erklärten Varianzverhältnisse anzeigen
	for i, ev in enumerate(pca.explained_variance_ratio_):
		ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

	# Einen verketteten DataFrame zurückgeben
	return pd.concat([variance_ratios, components], axis = 1)

In [ ]:
pca_results = pca_results(good_data, pca)

In [ ]:
# PCA mit nur zwei Dimensionen an die guten Daten anpassen
pca = PCA(n_components=2,random_state=0)
pca.fit(good_data)

# eine PCA-Transformation der guten Daten anwenden
reduced_data = pca.transform(good_data)

# eine PCA-Transformation auf die drei Stichproben-Datenpunkte anwenden
pca_samples = pca.transform(log_samples)

# einen DataFrame für die reduzierten Daten erstellen
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])
#display(reduced_data)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
fig.suptitle('Scatterplot of First Two Principal Components', fontsize=15)
img = ax.scatter(reduced_data["Dimension 1"], reduced_data["Dimension 2"],
                 c="#9A0EEA", s=100, alpha=0.4, linewidths=0)

## 2.2 Modellierung
Eine wichtige Frage ist es, welche Clustering-Methode geeignet ist. Hier ist das Gaußsische Mischungsmodell ausgewählt, da es uns aufgefallen ist, dass die Daten eine Tendenz zur Überlappung haben. Daher ist GMM die beste Option.

## Cluster erzeugen

In [ ]:
# GMM und silhouette_score importieren
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

# Unterschiedliche Werte für die Anzahl der zu verwendenden Cluster (von 2 bis 11)
num_clusters_list = range(2, 11+1)
num_cluster_values = len(num_clusters_list)

# Initialisieren die Listen, um Vorhersagen, Schwerpunkte und Bewertungspunkte zu speichern
preds_list = [[]] * num_cluster_values         # Predictions
sample_preds_list = [[]] * num_cluster_values  # Predictions for sample data
centers_list = [0] * num_cluster_values        # Centers
score_list = [0] * num_cluster_values          # scores

# For each value of clusters to consider, perform clustering and make
# Vorhersagen, die Zentren speichern und die Punktzahl berechnen
for i, num_clusters  in enumerate(num_clusters_list):
    # den Clustering-Algorithmus auf die reduzierten Daten anwenden
    clusterer = GaussianMixture(n_components=num_clusters, covariance_type='diag', random_state=4)
    clusterer.fit(reduced_data)

    # Vorhersage des Clusters für jeden Datenpunkt
    preds_list[i] = clusterer.predict(reduced_data)

    # die Clusterzentren finden
    centers_list[i] = clusterer.means_

    # Vorhersage des Clusters für jeden transformierten Stichproben-Datenpunkte
    sample_preds_list[i] = clusterer.predict(pca_samples)

    # den mittleren Silhouetteskoeffizienten für die Anzahl der ausgewählten Cluster berechnen
    score_list[i] = silhouette_score(reduced_data, preds_list[i],
                                     metric='euclidean', sample_size=None,
                                     random_state=4)

In [ ]:
import operator

# Holt den k-Wert, der zum besten Clustering-Modell führte
best_k_index = max(enumerate(score_list), key=operator.itemgetter(1))[0]
k = num_clusters_list[best_k_index]

# Holt die Vorhersagen, Zentren und Punkte für das beste Clustering-Modell
preds = preds_list[best_k_index]
centers = centers_list[best_k_index]
score = score_list[best_k_index]

print("Bestes Ergebnis kann es erreichen wenn k={} (score={:0.3f})".format(k, score))

In [ ]:
# Farbpalette für Clusterpunkte
PALLETTE =  ["#FF8000", "#5BA1CF", "#9621E2", "#FF4F40", "#73AD21",
            "#FFEC48", "#DE1BC2", "#29D3D1", "#B4F924", "#666666", "#AF2436"]

# Für jeden der Anzahl der berücksichtigten Clusterwerte ein Unterdiagramm erstellen
f, axes = plt.subplots(int(np.ceil(len(num_clusters_list) / 2.0)), 2,
                       figsize=(10, 14), sharex=True, sharey=True)
axes = [item for row in axes for item in row] # Unroll axes to a flat list

for i in range(len(num_clusters_list)):
    # Datenpunkte plotten, wobei je nach Clusterzuordnung eine andere Farbe zugewiesen wird
    ax = axes[i]
    colors  = [PALLETTE[y] for y in preds_list[i]]
    ax.scatter(reduced_data["Dimension 1"],
               reduced_data["Dimension 2"],
               c=colors, s=100, alpha=0.4, linewidths=0)

    # den Titel für den Nebenplot festlegen, einschließlich der Anzahl der verwendeten Cluster und
    # die Silhouetten-Punktzahl.
    ax.set_title(
        "K = {k}    (Silhouettenkoeffizieten = {score:.3f})"\
        "".format(k=num_clusters_list[i], score=score_list[i]),
        fontdict= {"style": "italic", "size": 10})

# den Titel für die Figur festlegen
t = f.suptitle('Ergebnissen des Clusterings (and Silhouettenkoeffizienten) mithilfe GMM',
               fontsize=15,
               fontdict={"fontweight": "extra bold"})

In [ ]:
def cluster_viz(reduced_data, labels, centers=None, reduced_samples=None, title="", legend_labels=["Segment 1","Segment 2"]):
   
    f, ax = plt.subplots(1, 1,  figsize=(12, 10))

    PALLETTE =  ["#FF8000", "#5BA1CF"]
    colors  = [PALLETTE[y] for y in labels]

    classes = np.unique(labels)
    for class_id in classes:
        ax.scatter(reduced_data["Dimension 1"][labels==class_id],
                   reduced_data["Dimension 2"][labels==class_id],
                   label=legend_labels[class_id],
                   c=PALLETTE[class_id], s=100, alpha=0.4, linewidths=0)

#     ax.scatter(reduced_data["Dimension 1"][labels==class_id],
#        reduced_data["Dimension 2"][labels==class_id],
#        c=colors, s=100, alpha=0.4, linewidths=0)


    # Mittelpunkte plotten
    if centers is not None:
        for i, c in enumerate(centers):
            ax.scatter(x = c[0], y = c[1], color = 'white', edgecolors=PALLETTE[i], \
                       alpha=1, linewidth=2, marker = 'o', s=300);
            ax.scatter(x = c[0], y = c[1], marker='${}$'.format(i), alpha=1, s=100);

    # die Stichprobe-Datenpunkte plotten
    if reduced_samples is not None:
        ax.scatter(x = reduced_samples[:,0], y = reduced_samples[:,1], \
                   s = 300,
                   linewidth = 2,
                   color = 'black',
                   facecolors = 'none',
                   edgecolors='black',
                   marker = 'o');

        for i in range(len(reduced_samples)):
            ax.scatter(x = reduced_samples[i,0]+0.4, y = reduced_samples[i,1], marker='$({})$'.format(i), alpha = 1, color='black', s=350);

    ax.legend(loc="lower right", frameon=False)

    # Titel des Plots festlegen
    ax.set_title(title);

In [ ]:
cluster_viz(reduced_data,
            labels=preds,
            centers=centers,
            reduced_samples=pca_samples,
            title="Customer Segments Learned by Model on PCA-Reduced Data")

## Zentroid berechnen

In [ ]:
# die Inversion der Transformation der Zentren
log_centers = pca.inverse_transform(centers)

# die Zentren potenzieren (Umkehrung der Log-Transformation)
true_centers = np.exp(log_centers)

# die wahren Zentrenn anzeigen
segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(true_centers), columns = data.keys())
true_centers.index = segments
display(true_centers)

In [ ]:
(true_centers - data.mean())

In [ ]:
sample_preds = sample_preds_list[best_k_index]

# die Vorhersagen ausdrucken
potential_cust_segments = ["Einzelhandel", "Restaurant"]
for i, pred in enumerate(sample_preds):
    print("Client {} predicted to be in Segment {} ({})".format(i, pred,
                                                 potential_cust_segments[pred]))

In [ ]:
(true_centers - data.mean()) / data.std()

## Wiedereinführung des Merkmals *Channel* in den Datensatz

In [ ]:
# die vollständigen Daten laden
try:
    full_data = pd.read_csv("../input/wholesale-customers-data-set/Wholesale customers data.csv")
except:
    print("Dataset could not be loaded.")

In [ ]:
def channel_results(reduced_data, outliers, pca_samples):


	# Überprüfen,ob das Dataset ladbar ist.
	try:
	    full_data = pd.read_csv("../input/wholesale-customers-data-set/Wholesale customers data.csv")
	except:
	    print("Dataset could not be loaded. Is the file missing?")       
	    return False

	# den Channel-DataFrame erstellen
	channel = pd.DataFrame(full_data['Channel'], columns = ['Channel'])
	channel = channel.drop(channel.index[outliers]).reset_index(drop = True)
	labeled = pd.concat([reduced_data, channel], axis = 1)
	
	# das Cluster-Plot generieren
	fig, ax = plt.subplots(figsize = (14,8))

	# Farbkarte
	cmap = cm.get_cmap('gist_rainbow')

	# die Punkte basierend auf dem zugewiesenen Kanal färben
	labels = ['Hotel/Restaurant/Cafe', 'Retailer']
	grouped = labeled.groupby('Channel')
	for i, channel in grouped:   
	    channel.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', \
	                 color = cmap((i-1)*1.0/2), label = labels[i-1], s=30);
	    
	# die transformierte Datenpunkte plotten 
	for i, sample in enumerate(pca_samples):
		ax.scatter(x = sample[0], y = sample[1], \
	           s = 200, linewidth = 3, color = 'black', marker = 'o', facecolors = 'none');
		ax.scatter(x = sample[0]+0.25, y = sample[1]+0.3, marker='$%d$'%(i), alpha = 1, s=125);

	# Set plot title
	ax.set_title("PCA-Reduced Data Labeled by 'Channel'\nTransformed Sample Data Circled");

In [ ]:
# die Clustering-Ergebnisse basierend auf "Channel"-Daten anzeigen
channel_results(reduced_data, outliers, pca_samples)

## 3. Projekterweiterung
Vorhersagen von Segmentierung der Neukunden durch die logistische Regression.
* **Problemstellung**: Angenommen, wir haben 10 neue Kunden. Zu welchem Segment gehören sie? 
* **Erklärende Variablen**: Fresh, Milk, Grocery, Detergents_Paper, Delicatessen
* **Zielvariablen**: Segment

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# Daten auf den ursprünglichen Zustand transformieren
data_preds = preds_list[best_k_index]

new_data = np.exp(good_data)

new_data = pd.DataFrame(new_data, columns = data.keys())
segment = preds_list[best_k_index]

# Ein neues Merkmal hinzufügen
new_data['Segment'] = segment

# Neuer Datensatz sieht so aus
new_data.head()

In [ ]:
#Datensatz in der Merkmale und Zielvariable aufteilen
feature_cols = ['Fresh', 'Milk', 'Grocery', 'Frozen','Detergents_Paper','Delicatessen']
X = new_data[feature_cols] # erklärende Variablen
y = new_data.Segment # Zielvariablen

In [ ]:
# X und y in Trainings- und Testsätze aufteilen (25-75)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
# Instanziieren des Modells (mit den Standardparametern)
# Das neue Model definieren
lr_model = LogisticRegression()

# das Modell mit Daten anpassen
lr_model.fit(X_train,y_train)

# Mit dem Testingdatensatz testen
y_pred=lr_model.predict(X_test)

In [ ]:
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a.head()

In [ ]:
# Validierung mit der Konfusionsmatrix
# die Metrikklasse importieren
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
class_names=[0,1] #Name der Klasse
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# Heatmap erstellen
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

## Ergebnisse

In [ ]:
# Angenommen, die neuen Kunden haben folgende Ausgaben
new_customer = {'Fresh': [15000,22276,7000,30500,11020,1000,9858,34000,9621,25000],
                  'Milk': [4312,2000,3353,9831,5100,5000,12000,11500,1373,3470],
                  'Grocery': [11008,9876,43245,11246,5611,14100,2200,23000,985,18000],
                'Frozen': [9882,3712,631,16500,5686,800,230,18800,8530,6200],
                'Detergents_Paper': [1451,4231,3256,1221,5545,2500,650,1609,980,3800],
                'Delicatessen': [400,1001,3534,9832,10000,800,916,1800,60,2489]
                  }

predict = pd.DataFrame(new_customer,columns= ['Fresh', 'Milk','Grocery','Frozen','Detergents_Paper','Delicatessen'])
y_pred = lr_model.predict(predict)
predict['Segment'] = y_pred
display(predict)

## Fazit: 

* Mithilfe der Clusteringsmethode können wir die Kunden im Großhandel in zwei Segmente klassifizieren.

* Nach dem Clustering kann man das Modell auf die Verkauf/Marketing/Lieferservice-Strategien verwenden.